<a href="https://colab.research.google.com/github/edKaleb/omdenaMexico/blob/main/mexico_night_time_lights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#https://developers.google.com/earth-engine/python_install
#You 1st need to sign up your Google account here: https://earthengine.google.com/ (it takes a couple of hours to get accepted. The API won't work before.)
import ee
import folium
from IPython.display import Image
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=OebQuwenedxMkbFGk_DDQc7UE8X_sG9cU3tVkoGlmmU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhA285rvPkihh29q_Awv7pTgkp2Bqjay3Fa_r_ugx6AUj5SXBBTXZE

Successfully saved authorization token.


# Trend Map
#### red = increase of light, blue = decrease of light, green = no change

In [ ]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, eeImageObject, visParams, name):
    mapID = ee.Image(eeImageObject).getMapId(visParams)
    folium.raster_layers.TileLayer(
    tiles = "https://earthengine.googleapis.com/map/"+mapID['mapid']+
      "/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
myMap = folium.Map(location=[23.67,-102.41], zoom_start=6, height=1000)

# Adds a band containing image date as years since 1991.
def createTimeBand(img):
    year = ee.Date(img.get('system:time_start')).get('year').subtract(1991)
    return ee.Image(year).byte().addBands(img)


# Map the time band creation helper over the night-time lights collection.
# https://developers.google.com/earth-engine/datasets/catalog/NOAA_DMSP-OLS_NIGHTTIME_LIGHTS
collection = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS').select('stable_lights').map(createTimeBand)

# Compute a linear fit over the series of values at each pixel, visualizing
# the y-intercept in green, and positive/negative slopes as red/blue.
myMap.add_ee_layer(collection.reduce(ee.Reducer.linearFit()),{'bands':['scale', 'offset', 'scale'], 'min': 0, 'max': [0.18, 20, -0.18]},'stable lights trend')

# Add a layer control panel to the map.
myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)

# Light vs no Light

In [32]:
collection = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').select('avg_rad')
collection = collection.filterDate('2018-09-01', '2019-09-30')

myMap = folium.Map(location=[23.67,-102.41], zoom_start=5, height=600)

myMap.add_ee_layer(collection.reduce(ee.Reducer.median()),
                   {'min': 0, 'max': 20},
                   'average radiation')

# Add a layer control panel to the map.
myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)

# Only apply mask on Mexico


In [33]:
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
myMap = folium.Map(location=[23.67,-102.41], zoom_start=6, height=1000)

mexico = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Mexico'))
  
def clip_dataset(image):
    return image.clip(mexico)


dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG').filter(ee.Filter.date('2018-09-01', '2019-09-30')).map(clip_dataset).max()
nighttimeLights = dataset.select('avg_rad')
# nighttimeLightsImage = nighttimeLights.reduce(ee.Reducer.max()).clipToCollection(mexico)

nighttimeLightsImage = nighttimeLights.clipToCollection(mexico)

nighttimeLightsVis = {
  min: 0.5,
  max: 1.0,
}

# Compute a linear fit over the series of values at each pixel, visualizing
# the y-intercept in green, and positive/negative slopes as red/blue.
myMap.add_ee_layer(nighttimeLights, nighttimeLightsVis, 'Nighttime Lights median')

# Add a layer control panel to the map.
myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)